In [ ]:
def dataFile(file, firstSplit, arrayName, firstValue = False, secondValue = False):
    for line in file:
        aux = []
        linha = line.split(firstSplit)
        if (firstSplit == ':'):
            arrayName.append(linha[0])
            if (secondValue is not False):
                quebra = linha[1].split(',')
                firstValue.append(int(quebra[0]))
                secondValue.append(int(quebra[1]))
            else:
                firstValue.append(int(linha[1]))
        else:
            for x in range(len(linha)):
                aux.append(int(linha[x]))
            arrayName.append(aux)

In [ ]:
from pulp import*

# Conjuntos

H     = []         # Horarios           Conjunto de 36 horários de aula (Seg a Sab)
D     = []         # Dias               Conjunto de dias de aula (0-5)
P     = []         # Docentes           Conjunto de 5 docentes
T     = []         # Turmas             Conjunto de 9 turmas
Sp    = []         # Preferencias       Conjunto de 4 turmas de preferência do docente p ∈ P (Sp ⊂ P)

# Parâmetros

Cht   = []         # Cht                Conjunto de carga horária das disciplinas
Cmin  = []         # Cmin               Conjunto de carga horária mínima dos docentes
Cmax  = []         # Cmax               Conjunto de carga horária máxima dos docentes
aulas = []         # Aulas              Conjunto de dias de aula de acordo com a disciplina

# Leitura

D = [1, 2, 3, 4, 5, 6]

file = open('slotes.txt', encoding='utf-8', mode='r')
dataFile(file, ",", H)
file.close()

file = open('docentes.txt', encoding='utf-8', mode='r')
dataFile(file, ":", P, Cmin, Cmax)
file.close()

file = open('turmas.txt', encoding='utf-8', mode='r')
dataFile(file, ":", T, Cht)
file.close()

file = open('preferencias.txt', encoding='utf-8', mode='r')
dataFile(file, ",", Sp)
file.close()

file = open('horarios.txt', encoding='utf-8', mode='r')
dataFile(file, ",", aulas)
file.close()

In [ ]:
print('\nDias\n')
print(D)

print('\nSlotes\n')
print(H)

print('\nDocentes\n')
print(P)

print('\nCarga horária mínima\n')
print(Cmin)

print('\nCarga horária máxima\n')
print(Cmax)

print('\nTurmas\n')
print(T)

print('\nCarga horária das turmas\n')
print(Cht)

print('\nPreferências\n')
print(Sp)

print('\nAulas\n')
print(aulas)

In [ ]:
# Construção do alpha

alpha = []

for t in range(len(T)):
    alpha.append([])
    for d in range(len(D)):
        alpha[t].append([])
        for h in range(len(H)):
            alpha[t][d].append(0)

In [ ]:
# Edição do alpha

for t in range(len(T)):
    for h in range(len(H)):
        if aulas[t][h] > 0:
            if aulas[t][h] < 6:
                d = 0
                s = aulas[t][h]
                alpha[t][d][s] = 1
            else:
                d = aulas[t][h]//6
                s = (aulas[t][h] - (6*d))
                alpha[t][d][s] = 1

In [ ]:
print(alpha)

In [ ]:
# Criação de dicts para variáveis específicas

Pref = makeDict([P], Sp, 0)

print('\nPares de preferências')
print(Pref)

In [ ]:
prob = LpProblem("Atribuição_de_Turmas",LpMinimize)

In [ ]:
# Decisão

x = LpVariable.dicts("x",(P, range(0, len(T))), 0, 1, cat='Binary')
    
prob += lpSum([int(Pref[p][t])*x[p][t] for t in range(len(T)) for p in P])

In [ ]:
# Toda turma tem docente

for t in range(len(T)):
    prob += lpSum([x[p][t] for p in P if Pref[p][t]>0]) == 1

In [ ]:
# Respeita a carga horária máxima de cada docente

for p, m in zip(P, range(len(P))):
    prob += lpSum([Cht[t]*x[p][t] for t in range(len(T)) if Pref[p][t]>0]) <= Cmax[m]

In [ ]:
# Respeita a carga horária mínima de cada docente

for p, m in zip(P, range(len(P))):
    prob += lpSum([Cht[t]*x[p][t] for t in range(len(T)) if Pref[p][t]>0]) >= Cmin[m]

In [ ]:
# Impede choque de horário

for p in P:
    for d in range(len(D)):
        for h in range(len(H)):
            prob += lpSum([alpha[t][d][h]*x[p][t] for t in range(len(T)) if Pref[p][t]>0]) <= 1

In [ ]:
print(prob)

In [ ]:
# Solucionar p problema

status = prob.solve()

# Status de solução

if status == LpStatusInfeasible:
    print('Modelo inviável!')
elif status == LpStatusUnbounded:
    print('Solução ilimitada!')
elif status == LpStatusOptimal:
    print('Solução ótima encontrada!')
else:
    print('Ops! Algo deu errado!')

In [ ]:
print('Solução:')
for i in prob.variables():
    if int(i.varValue)==1:
        print(i.name, "=", int(i.varValue))